***
# Trabajo Práctico 1: Demarco-Mendoza-Rossi
***

${\quad \quad \Large\textbf{1. Ganancias de cada etapa}}$

$$SNR_1 = \frac{h^2 \mathbb{E}[X^2_{1}]}{\sigma^2} = \frac{h^2 \mathbb{E}[X^2_{i+1}]}{\sigma^2} = \frac{h^2}{\sigma^2} \mathbb{E}\big[[G_{i+1}(h X_i + W_i)]\big]^2 = \frac{h^2}{\sigma^2} \big[\mathbb{E}[(hX_i)^2]+ \mathbb{E}[2(h \overbrace{Xi Wi}^{\text{Xi e Wi ind.}})] + \mathbb{E}[W^2_i]\big] = \frac{h^2}{\sigma^2} \big[h^2\mathbb{E}[X_i^2]+ \overbrace{2h\mathbb{E}[(Xi)]\underbrace{\mathbb{E}[(Wi)]}_{\text{ $= 0$}}}^{\text{ = 0}} + \mathbb{E}[W^2_i]\big] =  \frac{h^2}{\sigma^2} G^2_{i+1} \mathbb{E}\big[h^2 \xi + \sigma\big] = G^2_{i+1} \frac{1}{\sigma^2} [h^4\xi +\sigma] $$

$$SNR_1 = \frac{h^2 \xi}{\sigma}$$

$$G_{i+1} = \sqrt{\frac{\xi}{h^2 \xi + \sigma^2}}$$



${\quad \quad \Large\textbf{2.a) Salidas } Y_{n}\textbf{ en función de } W \textbf{, } X_1 \textbf{ y } G}$

Comenzando por la definición de $Y_{i} = h X_i + W_i$, con $X_i = G_i Y_{i-1}$, se desarrolla en forma genérica para un cierto $Y_n$, de forma que para $Y_5$ resulte:

$$ Y_5 = h \left(  G_5(h (G_4(h (G_3(h G_2(h X_1 + W_1) + W_2) + W_3) + W_4) + W_5 \right) $$

Que reescrito en una forma mas útil es:

$$ Y_5 = X_1 h^5 G_2 G_3 G_4 G_5 + \left[  W_1 (G_2 G_3 G_4 G_5) h^4 + W_2 (G_3 G_4 G_5) h^3 + W_3 (G_4 G_5) h^2 + W_4 (G_5) h + W_5 \right]  $$

De una forma genérica se puede expresar:

$$ Y_{i} = X_{1} h^{i} \prod_{k=2}^{i} G_{k} + \sum_{j=1}^{i} h^{i-j} W_{j+1} \left[ \prod_{k=j+1}^{i} G_{k} \right] $$

Las distribuciones del ruido son una normal de media 0 y de varianza (ver varianza de esto)